# Block 3: Wind Power Production in Real Life.


## Objectives
Welcome to the third and final block in the DIFUSE Wind Energy Module!

By the end of this block, you should be able to
- Recognize the limits of the wind power equations,
- Interpret a wind turbine's power curve,
- Define and calculate the capacity factor of a wind turbine, and
- Interact with regional wind speed data to calculate capacity factors for different turbines.









<font color = red> Before attempting *Block 3*, </font>
- Ensure you clear all outputs by going to *Edit --> Clear all outputs*
- **RUN**  </font> the code block below. 

*To run a code block, move the cursor to the left edge of the
block, and click the play button that appears at the top left.*

![picture](https://drive.google.com/uc?id=150ersKJY7dH-pDBkkHhCXtrmcljtiB9x)

In [ ]:
#@title
## PLEASE DO NOT EDIT THIS CODE BLOCK
## BLOCK 3 IMPLEMENTS

#importing necessary modules
import pandas as pd
import plotly.express as px
import numpy as np
from numpy import trapz

#import the csv containing the power curves data
dload = pd.read_csv("https://raw.githubusercontent.com/difuse-dartmouth/wind-speed-power-analysis/main/completed_module/data/Enercon_E-16.csv", skiprows = [0])
dload2 = pd.read_csv("https://raw.githubusercontent.com/difuse-dartmouth/wind-speed-power-analysis/main/completed_module/data/NPS100C-24.csv", skiprows=[0])

#import csv for Galveston, TX wind speed data
wind_data = pd.read_csv("https://raw.githubusercontent.com/difuse-dartmouth/wind-speed-power-analysis/main/completed_module/data/WindSpeedGalveston.csv")

#wind_data is sliced to only March 1st, 2021 Data
#Columns "HR" and "WS50M" are renamed to "Hour" and "Wind Speed (m/s)" 
day_speed = wind_data.iloc[54018:54042,:].rename(columns={'HR': 'Hour', 'WS50M': 'Wind Speed (m/s)'})

#wind_data is sliced to only 2021 Data
#Columns "HR" and "WS50M" are renamed to "Hour" and "Wind Speed (m/s)" 
last_year = wind_data.iloc[43818:52602,:].rename(columns={'HR': 'Hour', 'WS50M': 'Wind Speed (m/s)'})

#Function to read power value off respective power curves, given input wind speed
def speed_to_power_df(speed_df, curve_df):
  """Parameters:
  speed_df = (dataframe) WindSpeed Dataframe: Must contain only two columns - hour and windspeed
             Slice as needed from original dataframe "wind_data"
  curve_df = (dataframe) Power Curve dataframe: Must contain only two columns - windspeed and power
            Slice as needed from original dataframe "dload"
  --------------"""
  #empty windpower list
  wpower_list = []

  #index counter for speed dataframe
  count1 = -1

  for wspeed in speed_df.iloc[:,1]:
    count1 += 1

    #index counter for power curve dataframe
    count2 = -1
    for curve_speed in curve_df.iloc[:,0]:
      count2 += 1
      #reads corresponding power value from power curve dataframe
      #and appends it to windpower list
      if round(wspeed) == curve_speed:
        wpower = curve_df.iloc[count2,1]
        wpower_list.append([speed_df.iloc[count1,0],wpower])
  
  #makes and returns new dataframe of resulting power outputs plus corresponding times
  new_df = pd.DataFrame(wpower_list, columns = ["Hour", "Power (kW)"])
  return new_df

#Function to count frequency of wind speed values (to nearest whole number)
def freq_counter(speed_df, column_label):
  """Parameters:
  speed_df = (dataframe) WindSpeed Dataframe: Must contain a labelled column with wind_speed data 
  column_label = (string) label of the column with the wind speed data to be counted
  --------------"""
  #empty frequency dictionary
  freq_dict = {}

  #rounds speed value and counts it into dictionary
  for value in speed_df.loc[:,column_label]:
    if round(value) in freq_dict:
      freq_dict[round(value)] += 1
    else:
      freq_dict[round(value)] = 1
  
  #sorts frequency dictionary (orders wind speeds numerically)
  #creates and returns new dataframe of frequency table
  sorted_dict = sorted(freq_dict.items())
  new_df = pd.DataFrame(sorted_dict, columns = [column_label, "No. of hours"])
  return new_df



##**Power Curve**
The power output of a wind turbine is dependent upon the velocity of the wind that is hitting the blades. However, with onshore and offshore plants, the power is not proportional to the wind velocity. 

Every turbine is different, and while the wind power equation gives us a useful estimate of power production, in order to determine the output of a specific turbine at a given wind velocity, one needs its <u> *power curve*. 

![picture](https://drive.google.com/uc?id=1k3fL_bD66_mCjyf6wg2kR9HW__6lIoiC) 


As shown in the figure above, even though the <a href="https://en.wind-turbine-models.com/turbines/365-nps-northern-power-nps-100c-24" target="_blank">NPS 100C-24</a> is a 95 kW rated turbine, it only provides that power at a very limited range of wind speeds - about 12 m/s through 15 m/s.

\\
**What does the Wind Power Equation tell us?**

<font size = 4> $P = \frac{1}{2} \cdot \rho_{air} \cdot A \cdot v ^3$ </font> says that power production should always increase with increasing wind speeds.

***However*** according to NPS 100C-24's power curve,
*the power output starts to decrease beyond wind speeds of 15 m/s.*

\\
**Why?**

This is due to the deisgn specifications of the turbine. Also, for safety reasons, the turbine will stop spinning if the wind speed is higher than 25 m/s; this is called the **cut-out wind speed**.

\\
*NOTE: The Wind Power equation does not give us information about the optimal wind speeds for <u>each and every</u> turbine*




In [ ]:
#@title
fig = px.line(data_frame=dload2, x="Wind Speed (m/s)", y="Power (kW)", title="NPS 100C-24 Power Curve", width=850, height=400, markers=True)
fig.show()

###Concept Check 1
<font color = red> **RUN**</font> the code block above to generate the power curve for the NPS 100C-24 turbine. **What is the expected power output, in kW, for a wind speed of 8m/s?**

Using the Wind Power Equation, **what is the calculated power output at 8 m/s?**

$\rho_{air} = 1 \ kg/m^3$ and $A = 467.6 \ m^2$

<font size = 4> Recall, $P = \frac{1}{2} \cdot \rho_{air} \cdot A \cdot v ^3$ </font>

**Is there a difference between the values? Which value is greater?**


You should find the value from the wind power equation is much greater than the value from the power curve. This is because wind turbines are not 100% efficient and cannot collect all the energy from the wind.

Similar to the cut-off wind speed, turbines have a **cut-in wind speed** at which they can <u>start</u> harvesting wind energy. *On the power curve, wind speeds less than the cut-in wind speed yield 0 W of power.* 

Look again at the power curve for the NPS 100C-24 turbine, **what is its cut-in wind speed?**

##**Determining Actual Energy Output**
While wind turbines are rated by the maximum power they can produce (i.e. $95 \ kW$ for the NPS 100C-24), we can still determine the *actual* energy production of a wind turbine by using its power curve and the wind speed data of the region it is in.

**How?**

- We get the power output at a specific wind velocity from the power curve (i.e. $8 \ m/s$ yields $60 \ kW$ for the NPS 100C-24)
- We check raw speed data to get the number of hours the wind is blowing at that velocity. (Let's say the wind blows at 8m/s for 10 hours)
- Multiply power output by number of hours to get a value for wind energy produced \\
($60 \ kW \cdot 10$ hours$ = 600 \ kWh = 2160 \ MJ$ (Mega Joules)

Now, we will take a look at another power curve, belonging to the <a href="https://en.wind-turbine-models.com/turbines/605-enercon-e-16" target="_blank">*Enercon E-16*</a> turbine rated at $55 \ kW$


<font color = red> **RUN** the code block below to see the power curve</font>


In [ ]:
#@title
fig = px.line(data_frame=dload, x="Wind Speed (m/s)", y="Power (kW)", title="Enercon E-16 Power Curve", width=850, height=400, markers=True)
fig.show()

With a wind speed frequency table as seen in Block 2, we can get the number of hours the wind speed is blowing at a certain velocity.

In the code block below, we look at wind speed data in Galveston, Texas on March 1st, 2021

*NOTE: This wind speed data was collected at 50m.*

<font color = red> **RUN** the code block below to generate the frequency table</font>

<font color = blue> <i> - Sum up the number of hours in the frequency table. Ensure that it is 24.

In [ ]:
#@title
freq_table_march_1 = freq_counter(day_speed,"Wind Speed (m/s)")
freq_table_march_1

###Calculation
*Using the frequency table and the power curve above*, **calculate the actual energy output** of the Enercon E-16 for the entire day of March 1st, 2021 in Galveston, Texas.

What did you get? Does your answer match the one in the table below?
![picture](https://drive.google.com/uc?id=1dS-ZrTKtdyV-jYCgwRey5aI57MO-9utm)

$70 \ kWh$ ($252 \ MJ)$ is the energy used by an average U.S. household in just two days!

Alternatively, you could approach the problem visually using graphs. 

In the code block below, we have created a continous time plot of the Wind Speed in Galveston on March 1st, 2021

<font color = red> **RUN** the code block below to see the plot</font>

In [ ]:
#@title
fig = px.line(data_frame=day_speed, x="Hour", y="Wind Speed (m/s)", title="Wind Speed at 50m in Galveston TX on March 1st, 2021", width=850, height=400,markers = True)
fig.show()

Using the <a href="https://en.wind-turbine-models.com/turbines/605-enercon-e-16" target="_blank">*Enercon E-16*</a> power curve, we can create another graph of the corresponding power generated (kW) against the time (hours)

This graph is more useful because we know that the area under the curve would be $\text{Power } (kW)\cdot \text{Hour } (h) = \text{Energy produced }(kWh)$

Hence using integration (to find the area under the curve), we can also get the Energy produced.

*In fact, the initial method of using a table to solve for the energy utilizes the same principles as integration.*

<font color = red> **RUN** the code block below to see the final plot and resulting area</font>

In [ ]:
#@title
a = speed_to_power_df(day_speed.loc[:,["Hour","Wind Speed (m/s)"]], dload)
fig = px.line(data_frame=a, x="Hour", y="Power (kW)", title="Power generated with Enercon E-48 turbine over the day", width=850, height=400,markers=True)
fig.show()
y = a.iloc[:,1].values.tolist()
area = trapz(y, dx=1)
print("area under curve =", area)
print("The energy produced over the day is \033[1m\033[92m" + str(area) + "\033[0m kWh")

###Concept Check 2
Adding the area under the curve via integration yields an energy  output of $68.25 \ kWh \text{ or } 244.8 \ MJ$.

As said earlier, this can just power the average home for two days.

 Considering there are even more efficient wind turbines available, you can imagine how this value could be increased.

Now, compare this answer to the value gotten from the table method ($70 \ kWh \text{ or } 252 \ MJ$).

**Why are these values different? Which one do you think is more accurate?**

With both methods, we can get a good estimate of the energy produced over the day. However, using the integration method, we can better account for the continuous nature of wind speed values in real life (even though our wind speed data is discrete, taken every hour).




##**Capacity Factor**

This is the amount of energy generated by a turbine relative to the maximum amount of energy it could produce. This is expressed as a percentage, and is usually determined over the course of a <u> *single year*</u>.

$\text{Capacity factor} = \frac{\text{actual output}}{\text{maximum possible output}}$

So for the *Enercon E-16* with a power rating of 55kW, the maximum possible output for a year would be 

$55 \ kW \cdot 8760 \ \frac{hours}{year} = 481 \ 800 \ kWh/year$

Suppose, instead, that this turbine only generates $150 \ 000 \ kWh$ over the entire year.

**What is the capacity factor of this turbine?**

$\text{Capacity factor} = \frac{\text{actual output}}{\text{maximum possible output}} = \frac{150000}{481800} = 0.311 \text{ or } 31.1 \% $


###Task
With the following wind speed data, **calculate the capacity factor of the Enercon E-16 Turbine during the year 2020.**

*Remember that actual energy output is derived using the power curve and the number of hours at that power (i.e. wind speed)*

<font color =red> **RUN** the code block below to generate 2020 wind speed data for Galveston, TX

In [ ]:
#@title
#Wind Speeds for Gavalston, TX bin-sorted
fig = px.line(data_frame=dload, x="Wind Speed (m/s)", y="Power (kW)", title="Enercon E-16 Power Curve", width=850, height=400, markers=True)
fig.show()

windData = freq_counter(last_year,"Wind Speed (m/s)")
windData


###Concept Check 3
Using the table method, you should have gotten a yearly energy value of $11673 \ kWh$ and a capacity factor of $= \frac{\text{actual output}}{\text{maximum possible output}} = \frac{11673}{481800} = 0.024 \text{ or } 2.4 \% $

This is incredibly inefficient. The average capacity factor of a wind fleet in the U.S. is about 34%. This comparison tells us that the *Enercon E-16* is not the best turbine to use in a location such as Galveston, Texas.

Luckily, there are hundreds of wind turbines to choose from! Wind engineers can run the types of analyses described above to determine capacity factors of different turbines and select the best one for this particular area.

##**Wind Power Plant Placement Considerations**
We receive a lot of news on wind turbine and wind turbine efficiency. 

See this headline on <a href="https://www.keranews.org/energy-environment/2022-07-24/a-massive-wind-farm-project-is-being-proposed-along-the-coast-of-galveston/
" target="_blank">the development of wind energy in Galveston, Texas </a>
![picture](https://drive.google.com/uc?id=1sFPDTUEDEL83Uw8QmpWTBjGojff10ZPj)

This headline makes a huge claim: that the proposed plant could power 2.3 million homes. That is a big deal.

But how does one even come about a number like that?
**If you had to verify this headline, what sort of data would you need?**

From what we know so far, *to calculate the energy produced* you could use
- Wind speed data of the region
- The type of turbine(s) to be used and their power curves.

Finally, you could get the average energy comsumption of a home in Galveston and get a numerical estimate of the number of potential homes powered.

This is all a start, but in actuality much, *much* more data is needed to make a reasonable estimate on a project of this scale.

As you will learn in class, the **layout** of the wind farm greatly impacts its overall energy production. It dictates the spacing of the turbines and which direction they should face. So, you would also need the **wind direction** as well as its speed.

And these are just physical considerations. In the real world, wind farm planners must consider **cost**, **safety**, and **longterm efficiency** of the plant. 

In your class, outside of this data module, you will learn more about these placement considerations.



###Concept Check 4

The point of this Block is to introduce you to *some* of the real-world considerations in setting up a wind power plant. There is always a lot to consider long before construction begins.

With what you have learned, **list some limitations of the wind power equations in the siting of a wind power plant.**
**What factors are not taken into account in the equations?**

#Congratulations!
You have completed Block 3 of the module.

Ensure that you visit and understand all four concept checks in this Block before completing the module. 

Feel free to direct any questions to your professor.


#Credits
DIFUSE at Dartmouth
